In [1]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.functions import date_format
from pyspark.sql.functions import lit
import random
from pyspark.sql.types import *
from pyspark.sql.functions import upper
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import format_number
import pyspark.sql.functions as F

In [2]:
df_rtable = sqlContext.createDataFrame([("Green", "1", "1.3", "DescriptionONE"),
                                        ("Green", "0", "1.445", "DescriptionONE"),
                                        ("Blue", "4", "1.2", "DescriptionTWO"),
                                        ("Red", "5", "1.3", "Description THREE"),
                                        ("Yellow", "7", "1.325", "Description Four"), 
                                        ("Red", "9", "1.4", "DescriptionONE"),
                                        ("Red", "6", "1.72158", "Description THREE"),
                                        ("Blue", "5", "1", "DescriptionONE")],
                                       ("Input 1", "Input 2", "Input 3", "Description"))

In [3]:
def spacerem_upper(x):
    pos = x.find(' ')
    #print(pos)
    if pos != - 1:
        y = x[:pos] + x[pos+1 : ].upper()
        return y
    else:
        return x

spaceDeleteUDF = udf(spacerem_upper)
df_upper2 = df_rtable.withColumn("Description", spaceDeleteUDF("Description"))

In [4]:

df_upper3 =df_upper2.select(df_upper2['Input 1'], df_upper2['Input 2'], df_upper2['Input 3'].cast('float'), df_upper2['Description'])
df_upper4 = df_upper3.withColumn('Input 3',format_number(df_upper3['Input 3'],4))


In [5]:
df_rtable2 = sqlContext.createDataFrame([("Green", "Night"),
                                        ("Yellow", "Morning"),
                                        ("Red", "Afternoon"),
                                        ("Blue","Evening")])

In [6]:
def function_5_to_10(a,b):
    df_upper4  = a
    df_rtable2 = b
    
    df_rtable4 = df_rtable2.withColumnRenamed('_1','Input 1').withColumnRenamed('_2','Day Period')
    
    leftjoin = df_upper4.join(df_rtable4,['Input 1'],"left")
    
    def date_rand(a):
        date_YYYY = str(random.randint(1950, 2000))
        date_MM   = str(random.randint(01, 12)).zfill(02)
        date_DD   = str(random.randint(01, 28)).zfill(02)
        date_sep  = '-'
        return  date_YYYY + date_MM + date_DD 

    udfdate_rand = udf(date_rand, StringType())
    
    leftjoin = leftjoin.withColumn("Date",lit(0))
    
    leftjoin_w_date = leftjoin.withColumn("Date", udfdate_rand("Date"))
    
    leftjoin_w_date = leftjoin_w_date.withColumn("Input 3",leftjoin_w_date["Input 3"].cast('float'))
    
    df_calc2 =leftjoin_w_date.filter((leftjoin_w_date['Input 3'] < 1.31 ) 
                                 & (leftjoin_w_date['Input 1'] != 'Red') 
                                 & (leftjoin_w_date['Input 1'] != 'Green'))
    
    date_list = leftjoin_w_date.select('Date').collect()

    date_array = [int(i.Date) for i in date_list]
    
    defg = sorted(date_array)
    
    if len(defg)%2 == 0:
        date_cmp1 = defg[(len(defg)/2)-1]
        date_cmp2 = defg[(len(defg)/2)]
        if date_cmp1 == date_cmp2:
            date_cmp = date_cmp1
        else:
            date_cmp = date_cmp1
    else:
        date_cmp = defg[(len(defg)/2)-1]
        
    leftjoin_w_date = leftjoin_w_date.withColumn("Date",leftjoin_w_date["Date"].cast('int'))
    leftjoin_w_date = leftjoin_w_date.withColumn("Input 2",leftjoin_w_date["Input 2"].cast('int'))
    
    def flag_func(a,b):
        if a > date_cmp and b > 1:
            flag = 1 
        else:
            flag = 0 
        return flag
    
    udf_falg_func = udf(flag_func)
    
    leftjoin_w_flag = leftjoin_w_date.withColumn('flag', udf_falg_func("Date","Input 2"))
    
    df1 = leftjoin_w_flag.groupby('Description').agg(F.sum("Input 3")/(F.min("Input 2"))).alias('sumbymin')


In [7]:
a = function_5_to_10(df_upper4,df_rtable2)